# Production система поиска аналогов товаров

**Analog Search Production System**

Этот notebook загружает обученную систему поиска аналогов и создает финальную Excel таблицу с результатами анализа.

## Возможности:
- Загрузка обученной модели поиска аналогов
- Поиск аналогов для новых товаров
- Создание детального отчета в Excel формате
- Анализ и категоризация результатов

---

## 1. Настройка и импорты

In [1]:
# Системные импорты
import sys
import os
import warnings
import pandas as pd
import numpy as np
from pathlib import Path
from typing import List, Dict, Any, Tuple
import time
from datetime import datetime
import pickle
import json
from collections import defaultdict
import re

# Настройка отображения
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('display.width', None)

# Добавляем путь к модулям SAMe
sys.path.append(os.path.abspath('../../src'))
sys.path.append(os.path.abspath('../..'))

print("✅ Базовые импорты загружены")
print(f"📁 Рабочая директория: {os.getcwd()}")
print(f"🕐 Время запуска: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

✅ Базовые импорты загружены
📁 Рабочая директория: /Users/igor/Desktop/PythonProjects/SAMe/notebooks/production
🕐 Время запуска: 2025-07-18 15:41:50


In [2]:
# Импорты модулей SAMe
try:
    from same.data_manager import data_helper
    from same.text_processing.text_cleaner import TextCleaner, CleaningConfig
    from same.text_processing.lemmatizer import Lemmatizer, LemmatizerConfig
    from same.text_processing.normalizer import TextNormalizer, NormalizerConfig
    from same.text_processing.preprocessor import TextPreprocessor, PreprocessorConfig
    from same.search_engine.fuzzy_search import FuzzySearchEngine, FuzzySearchConfig
    from same.search_engine.semantic_search import SemanticSearchEngine, SemanticSearchConfig
    from same.search_engine.hybrid_search import HybridSearchEngine, HybridSearchConfig
    from same.parameter_extraction.regex_extractor import RegexParameterExtractor
    print("✅ Модули SAMe успешно загружены")
except ImportError as e:
    print(f"❌ Ошибка импорта модулей SAMe: {e}")
    print("💡 Убедитесь что модули созданы в директории src/same/")

✅ Модули SAMe успешно загружены


## 2. Загрузка обученной модели

In [3]:
# Загрузка обученной системы поиска аналогов
print("🔄 Загрузка обученной системы поиска...")

# Путь к моделям
models_dir = Path("../../models/analog_search")

# Определяем версию модели для загрузки
model_version = None

# Пробуем загрузить последнюю версию
latest_version_path = models_dir / "latest_version.txt"
if latest_version_path.exists():
    try:
        with open(latest_version_path, 'r') as f:
            model_version = f.read().strip()
        print(f"📋 Найдена последняя версия модели: {model_version}")
    except Exception as e:
        print(f"⚠️ Ошибка чтения версии: {e}")

# Если версия не найдена, ищем доступные модели
if not model_version:
    available_models = list(models_dir.glob("system_metadata_v*.json"))
    if available_models:
        # Берем последнюю по времени создания
        latest_model = max(available_models, key=lambda x: x.stat().st_mtime)
        model_version = latest_model.stem.replace("system_metadata_", "")
        print(f"📋 Найдена модель: {model_version}")
    else:
        print("❌ Обученные модели не найдены!")
        print("💡 Сначала запустите notebook product_analog_search.ipynb для обучения модели")
        raise FileNotFoundError("Модели не найдены")

print(f"🏷️ Используется версия модели: {model_version}")
print(f"📁 Директория моделей: {models_dir}")

🔄 Загрузка обученной системы поиска...
📋 Найдена последняя версия модели: v20250718_132401
🏷️ Используется версия модели: v20250718_132401
📁 Директория моделей: ../../models/analog_search


In [4]:
# Загрузка метаданных системы
metadata_path = models_dir / f"system_metadata_{model_version}.json"
try:
    with open(metadata_path, 'r', encoding='utf-8') as f:
        system_metadata = json.load(f)
    
    print("📊 Информация о модели:")
    print(f"   Версия: {system_metadata['version']}")
    print(f"   Создана: {system_metadata['created_at']}")
    print(f"   Записей в датасете: {system_metadata['dataset_info']['total_records']}")
    print(f"   Основной столбец: {system_metadata['dataset_info']['main_column']}")
    print(f"   Доступные движки: {', '.join(system_metadata['available_engines'])}")
    
except Exception as e:
    print(f"⚠️ Ошибка загрузки метаданных: {e}")
    system_metadata = {}

📊 Информация о модели:
   Версия: v20250718_132401
   Создана: 2025-07-18T13:24:01.547617
   Записей в датасете: 130303
   Основной столбец: Наименование
   Доступные движки: fuzzy, semantic, hybrid


In [5]:
# Загрузка компонентов предобработки
preprocessing_path = models_dir / f"preprocessing_{model_version}.pkl"
try:
    with open(preprocessing_path, 'rb') as f:
        preprocessing_components = pickle.load(f)
    
    # Извлекаем компоненты
    text_cleaner = preprocessing_components.get('text_cleaner')
    text_normalizer = preprocessing_components.get('text_normalizer')
    lemmatizer = preprocessing_components.get('lemmatizer')
    preprocessor = preprocessing_components.get('preprocessor')
    final_preprocess_function = preprocessing_components.get('final_preprocess_function')
    enhanced_simple_preprocess = preprocessing_components.get('enhanced_simple_preprocess')
    
    print("✅ Компоненты предобработки загружены:")
    print(f"   TextCleaner: {'✅' if text_cleaner else '❌'}")
    print(f"   TextNormalizer: {'✅' if text_normalizer else '❌'}")
    print(f"   Lemmatizer: {'✅' if lemmatizer else '❌'}")
    print(f"   TextPreprocessor: {'✅' if preprocessor else '❌'}")
    print(f"   Функции предобработки: {'✅' if final_preprocess_function else '❌'}")
    
    # Информация о функциях для отладки
    function_info = preprocessing_components.get('function_info', {})
    if function_info:
        print(f"\n📋 Информация о функциях предобработки:")
        print(f"   Метод сериализации: {function_info.get('serialization_method', 'unknown')}")
        print(f"   Оригинальная final_preprocess: {'✅' if function_info.get('has_original_final_preprocess') else '❌'}")
        print(f"   Оригинальная enhanced_simple: {'✅' if function_info.get('has_original_enhanced_simple') else '❌'}")
    
except Exception as e:
    print(f"❌ Ошибка загрузки компонентов предобработки: {e}")
    preprocessing_components = {}
    final_preprocess_function = None

❌ Ошибка загрузки компонентов предобработки: Can't get attribute 'enhanced_simple_preprocess' on <module '__main__'>


In [6]:
# Загрузка поисковых движков
engines_path = models_dir / f"search_engines_{model_version}.pkl"
try:
    with open(engines_path, 'rb') as f:
        available_engines = pickle.load(f)
    
    print("✅ Поисковые движки загружены:")
    for engine_name, engine in available_engines.items():
        engine_type = type(engine).__name__
        is_fitted = getattr(engine, 'is_fitted', False)
        print(f"   {engine_name}: {engine_type} {'✅' if is_fitted else '❌'}")
    
except Exception as e:
    print(f"❌ Ошибка загрузки поисковых движков: {e}")
    available_engines = {}

✅ Поисковые движки загружены:
   fuzzy: FuzzySearchEngine ✅
   semantic: SemanticSearchEngine ✅
   hybrid: HybridSearchEngine ✅


In [7]:
# Загрузка обработанных данных
data_path = models_dir / f"processed_data_{model_version}.pkl"
try:
    with open(data_path, 'rb') as f:
        processed_data = pickle.load(f)
    
    print(f"✅ Обработанные данные загружены:")
    print(f"   Записей: {len(processed_data)}")
    print(f"   Столбцов: {len(processed_data.columns)}")
    print(f"   Столбцы: {list(processed_data.columns)}")
    
    # Определяем основной столбец
    main_name_column = system_metadata.get('dataset_info', {}).get('main_column', 'Наименование')
    if main_name_column not in processed_data.columns:
        main_name_column = processed_data.columns[0]
    
    print(f"   Основной столбец: {main_name_column}")
    
except Exception as e:
    print(f"❌ Ошибка загрузки данных: {e}")
    processed_data = pd.DataFrame()
    main_name_column = 'Наименование'

✅ Обработанные данные загружены:
   Записей: 130303
   Столбцов: 7
   Столбцы: ['Наименование', 'processed_name', 'Код', 'НаименованиеПолное', 'Группа', 'ВидНоменклатуры', 'ЕдиницаИзмерения']
   Основной столбец: Наименование


## 3. Функции для поиска аналогов

In [8]:
# Функции для работы с системой поиска аналогов

def search_analogs(query: str, engine_name: str = 'hybrid', top_k: int = 10) -> List[Dict[str, Any]]:
    """
    Поиск аналогов для заданного запроса
    
    Args:
        query: Текст запроса
        engine_name: Название поискового движка ('fuzzy', 'semantic', 'hybrid')
        top_k: Количество результатов
    
    Returns:
        Список результатов поиска
    """
    if engine_name not in available_engines:
        print(f"⚠️ Движок '{engine_name}' недоступен. Доступные: {list(available_engines.keys())}")
        return []
    
    engine = available_engines[engine_name]
    
    try:
        # Предобработка запроса
        if final_preprocess_function:
            processed_query = final_preprocess_function(query)
        else:
            processed_query = query.lower().strip()
        
        # Поиск
        results = engine.search(processed_query, top_k=top_k)
        
        # Обогащение результатов данными из датасета
        enriched_results = []
        for result in results:
            if 'index' in result or 'document_id' in result:
                doc_id = result.get('index', result.get('document_id'))
                if doc_id < len(processed_data):
                    row = processed_data.iloc[doc_id]
                    enriched_result = {
                        **result,
                        'original_name': row.get(main_name_column, ''),
                        'processed_name': row.get('processed_name', ''),
                        'row_data': row.to_dict()
                    }
                    enriched_results.append(enriched_result)
        
        return enriched_results
        
    except Exception as e:
        print(f"❌ Ошибка поиска: {e}")
        return []

def analyze_similarity(query: str, candidate: str) -> Dict[str, Any]:
    """
    Анализ схожести между запросом и кандидатом
    
    Args:
        query: Исходный запрос
        candidate: Кандидат для сравнения
    
    Returns:
        Словарь с метриками схожести
    """
    from difflib import SequenceMatcher
    
    # Предобработка текстов
    if final_preprocess_function:
        query_processed = final_preprocess_function(query)
        candidate_processed = final_preprocess_function(candidate)
    else:
        query_processed = query.lower().strip()
        candidate_processed = candidate.lower().strip()
    
    # Различные метрики схожести
    sequence_similarity = SequenceMatcher(None, query_processed, candidate_processed).ratio()
    
    # Jaccard similarity (пересечение слов)
    query_words = set(query_processed.split())
    candidate_words = set(candidate_processed.split())
    
    if query_words or candidate_words:
        jaccard_similarity = len(query_words & candidate_words) / len(query_words | candidate_words)
    else:
        jaccard_similarity = 0.0
    
    # Определение типа связи
    if sequence_similarity > 0.95:
        relation_type = "дубль"
    elif sequence_similarity > 0.8:
        relation_type = "близкий аналог"
    elif sequence_similarity > 0.6:
        relation_type = "аналог"
    elif jaccard_similarity > 0.5:
        relation_type = "похожий товар"
    else:
        relation_type = "возможный аналог"
    
    return {
        'sequence_similarity': sequence_similarity,
        'jaccard_similarity': jaccard_similarity,
        'relation_type': relation_type,
        'similarity_score': max(sequence_similarity, jaccard_similarity)
    }

print("✅ Функции поиска аналогов определены")

✅ Функции поиска аналогов определены


## 4. Создание Excel отчета с анализом аналогов

In [9]:
# Функция для создания детального Excel отчета

def create_analog_analysis_report(sample_size: int = 1000, output_path: str = None) -> str:
    """
    Создает детальный Excel отчет с анализом аналогов
    
    Args:
        sample_size: Количество товаров для анализа (для ускорения)
        output_path: Путь для сохранения файла
    
    Returns:
        Путь к созданному файлу
    """
    print(f"📊 Создание отчета по анализу аналогов...")
    print(f"📝 Анализируем {sample_size} товаров из {len(processed_data)}")
    
    # Подготовка данных для анализа
    if sample_size < len(processed_data):
        # Берем случайную выборку
        sample_data = processed_data.sample(n=sample_size, random_state=42)
        print(f"📋 Используется случайная выборка: {len(sample_data)} записей")
    else:
        sample_data = processed_data.copy()
        print(f"📋 Используется полный датасет: {len(sample_data)} записей")
    
    # Структура результирующей таблицы
    results = []
    
    # Определяем движок для поиска (приоритет: hybrid > semantic > fuzzy)
    search_engine = None
    if 'hybrid' in available_engines:
        search_engine = 'hybrid'
    elif 'semantic' in available_engines:
        search_engine = 'semantic'
    elif 'fuzzy' in available_engines:
        search_engine = 'fuzzy'
    
    if not search_engine:
        print("❌ Поисковые движки недоступны!")
        return None
    
    print(f"🔍 Используется поисковый движок: {search_engine}")
    
    # Обработка каждого товара
    start_time = time.time()
    
    for idx, (_, row) in enumerate(sample_data.iterrows()):
        if idx % 100 == 0 and idx > 0:
            elapsed = time.time() - start_time
            rate = idx / elapsed
            eta = (len(sample_data) - idx) / rate if rate > 0 else 0
            print(f"📈 Обработано: {idx}/{len(sample_data)} ({idx/len(sample_data)*100:.1f}%) | ETA: {eta:.0f} сек")
        
        original_name = row[main_name_column]
        processed_name = row.get('processed_name', '')
        
        # Предобработка наименования
        if final_preprocess_function:
            try:
                cleaned_name = final_preprocess_function(original_name)
                # Попробуем получить промежуточные результаты
                if text_cleaner:
                    cleaned_only = text_cleaner.clean_text(original_name)
                else:
                    cleaned_only = original_name
                
                if lemmatizer:
                    try:
                        lemmatized_name = lemmatizer.lemmatize_text(cleaned_only)
                    except:
                        lemmatized_name = cleaned_only
                else:
                    lemmatized_name = cleaned_only
                
                normalized_name = cleaned_name
                
            except Exception as e:
                cleaned_name = original_name.lower().strip()
                lemmatized_name = cleaned_name
                normalized_name = cleaned_name
        else:
            cleaned_name = original_name.lower().strip()
            lemmatized_name = cleaned_name
            normalized_name = cleaned_name
        
        # Поиск аналогов
        try:
            analogs = search_analogs(original_name, search_engine, top_k=5)
            
            if analogs:
                # Берем лучший аналог (исключая сам товар)
                best_analog = None
                for analog in analogs:
                    if analog.get('original_name', '') != original_name:
                        best_analog = analog
                        break
                
                if best_analog:
                    candidate_name = best_analog.get('original_name', '')
                    
                    # Анализ схожести
                    similarity_analysis = analyze_similarity(original_name, candidate_name)
                    
                    # Определение рекомендованной категории
                    original_category = row.get('Группа', row.get('ВидНоменклатуры', ''))
                    candidate_row_data = best_analog.get('row_data', {})
                    candidate_category = candidate_row_data.get('Группа', candidate_row_data.get('ВидНоменклатуры', ''))
                    
                    if original_category != candidate_category and candidate_category:
                        suggested_category = candidate_category
                    else:
                        suggested_category = original_category
                    
                    # Определение финального решения
                    similarity_score = similarity_analysis['similarity_score']
                    if similarity_score > 0.95:
                        final_decision = "Дубликат - объединить"
                    elif similarity_score > 0.8:
                        final_decision = "Близкий аналог - проверить"
                    elif similarity_score > 0.6:
                        final_decision = "Аналог - рассмотреть замену"
                    else:
                        final_decision = "Требует ручной проверки"
                    
                    # Комментарий системы
                    comment_parts = []
                    if similarity_score > 0.9:
                        comment_parts.append("Высокая схожесть")
                    if original_category != candidate_category:
                        comment_parts.append(f"Разные категории: {original_category} vs {candidate_category}")
                    
                    search_score = best_analog.get('combined_score', best_analog.get('similarity_score', 0))
                    comment_parts.append(f"Поисковый скор: {search_score:.3f}")
                    
                    comment = "; ".join(comment_parts) if comment_parts else "Автоматический анализ"
                    
                else:
                    # Нет подходящих аналогов
                    candidate_name = "Аналоги не найдены"
                    similarity_analysis = {'similarity_score': 0, 'relation_type': 'нет аналогов'}
                    suggested_category = row.get('Группа', row.get('ВидНоменклатуры', ''))
                    final_decision = "Уникальный товар"
                    comment = "Аналоги не найдены в базе"
            
            else:
                # Поиск не дал результатов
                candidate_name = "Поиск не дал результатов"
                similarity_analysis = {'similarity_score': 0, 'relation_type': 'ошибка поиска'}
                suggested_category = row.get('Группа', row.get('ВидНоменклатуры', ''))
                final_decision = "Требует ручной проверки"
                comment = "Ошибка поиска или уникальный товар"
        
        except Exception as e:
            # Ошибка поиска
            candidate_name = f"Ошибка: {str(e)[:50]}"
            similarity_analysis = {'similarity_score': 0, 'relation_type': 'ошибка'}
            suggested_category = row.get('Группа', row.get('ВидНоменклатуры', ''))
            final_decision = "Ошибка анализа"
            comment = f"Ошибка поиска: {str(e)[:100]}"
        
        # Формирование записи результата
        result_row = {
            'Raw_Name': original_name,
            'Cleaned_Name': cleaned_name,
            'Lemmatized_Name': lemmatized_name,
            'Normalized_Name': normalized_name,
            'Candidate_Name': candidate_name,
            'Similarity_Score': f"{similarity_analysis['similarity_score']:.3f}",
            'Relation_Type': similarity_analysis['relation_type'],
            'Suggested_Category': suggested_category,
            'Final_Decision': final_decision,
            'Comment': comment,
            # Дополнительные поля для анализа
            'Original_Category': row.get('Группа', row.get('ВидНоменклатуры', '')),
            'Original_Code': row.get('Код', ''),
            'Search_Engine': search_engine
        }
        
        results.append(result_row)
    
    # Создание DataFrame
    results_df = pd.DataFrame(results)
    
    print(f"\n✅ Анализ завершен: {len(results_df)} записей обработано")
    
    return results_df

print("✅ Функция создания отчета определена")

✅ Функция создания отчета определена


In [10]:
# Создание отчета
print("🚀 Запуск создания отчета по аналогам...")

# Настройки анализа
SAMPLE_SIZE = 1000  # Количество товаров для анализа (измените по необходимости)
OUTPUT_DIR = Path("../../data/output")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Создание отчета
if len(processed_data) > 0 and available_engines:
    results_df = create_analog_analysis_report(sample_size=SAMPLE_SIZE)
    
    if results_df is not None and len(results_df) > 0:
        # Создание Excel файла с несколькими листами
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        excel_path = OUTPUT_DIR / f"analog_analysis_report_{timestamp}.xlsx"
        
        with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
            # Основной отчет
            results_df.to_excel(writer, sheet_name='Analog_Analysis', index=False)
            
            # Статистика по типам связей
            relation_stats = results_df['Relation_Type'].value_counts().reset_index()
            relation_stats.columns = ['Relation_Type', 'Count']
            relation_stats['Percentage'] = (relation_stats['Count'] / len(results_df) * 100).round(2)
            relation_stats.to_excel(writer, sheet_name='Relation_Statistics', index=False)
            
            # Статистика по решениям
            decision_stats = results_df['Final_Decision'].value_counts().reset_index()
            decision_stats.columns = ['Final_Decision', 'Count']
            decision_stats['Percentage'] = (decision_stats['Count'] / len(results_df) * 100).round(2)
            decision_stats.to_excel(writer, sheet_name='Decision_Statistics', index=False)
            
            # Топ аналогов по схожести
            top_similarities = results_df[results_df['Similarity_Score'] != '0.000'].copy()
            top_similarities['Similarity_Score_Numeric'] = pd.to_numeric(top_similarities['Similarity_Score'])
            top_similarities = top_similarities.nlargest(100, 'Similarity_Score_Numeric')
            top_similarities.drop('Similarity_Score_Numeric', axis=1).to_excel(writer, sheet_name='Top_Similarities', index=False)
            
            # Метаданные
            metadata_df = pd.DataFrame([
                ['Дата создания', datetime.now().strftime('%Y-%m-%d %H:%M:%S')],
                ['Версия модели', model_version],
                ['Поисковый движок', search_engine if 'search_engine' in locals() else 'N/A'],
                ['Всего записей в базе', len(processed_data)],
                ['Проанализировано записей', len(results_df)],
                ['Найдено аналогов', len(results_df[results_df['Candidate_Name'] != 'Аналоги не найдены'])],
                ['Средняя схожесть', results_df[results_df['Similarity_Score'] != '0.000']['Similarity_Score'].astype(float).mean()]
            ], columns=['Параметр', 'Значение'])
            metadata_df.to_excel(writer, sheet_name='Metadata', index=False)
        
        print(f"\n🎉 Excel отчет создан: {excel_path}")
        print(f"📊 Размер файла: {excel_path.stat().st_size / (1024*1024):.1f} MB")
        
        # Краткая статистика
        print(f"\n📋 Краткая статистика:")
        print(f"   Всего проанализировано: {len(results_df)}")
        print(f"   Найдено аналогов: {len(results_df[results_df['Candidate_Name'] != 'Аналоги не найдены'])}")
        print(f"   Дубликатов: {len(results_df[results_df['Relation_Type'] == 'дубль'])}")
        print(f"   Близких аналогов: {len(results_df[results_df['Relation_Type'] == 'близкий аналог'])}")
        print(f"   Обычных аналогов: {len(results_df[results_df['Relation_Type'] == 'аналог'])}")
        
        # Сохранение также в CSV для удобства
        csv_path = OUTPUT_DIR / f"analog_analysis_report_{timestamp}.csv"
        results_df.to_csv(csv_path, index=False, encoding='utf-8-sig')
        print(f"📄 CSV файл создан: {csv_path}")
        
    else:
        print("❌ Не удалось создать отчет")
else:
    print("❌ Недостаточно данных для создания отчета")
    print(f"   Данные загружены: {'✅' if len(processed_data) > 0 else '❌'}")
    print(f"   Движки доступны: {'✅' if available_engines else '❌'}")

🚀 Запуск создания отчета по аналогам...
📊 Создание отчета по анализу аналогов...
📝 Анализируем 1000 товаров из 130303
📋 Используется случайная выборка: 1000 записей
🔍 Используется поисковый движок: hybrid
❌ Ошибка поиска: 'SemanticSearchEngine' object has no attribute '_search_cache'
❌ Ошибка поиска: 'SemanticSearchEngine' object has no attribute '_search_cache'
❌ Ошибка поиска: 'SemanticSearchEngine' object has no attribute '_search_cache'
❌ Ошибка поиска: 'SemanticSearchEngine' object has no attribute '_search_cache'
❌ Ошибка поиска: 'SemanticSearchEngine' object has no attribute '_search_cache'
❌ Ошибка поиска: 'SemanticSearchEngine' object has no attribute '_search_cache'
❌ Ошибка поиска: 'SemanticSearchEngine' object has no attribute '_search_cache'
❌ Ошибка поиска: 'SemanticSearchEngine' object has no attribute '_search_cache'
❌ Ошибка поиска: 'SemanticSearchEngine' object has no attribute '_search_cache'
❌ Ошибка поиска: 'SemanticSearchEngine' object has no attribute '_search_ca

## 5. Интерактивный поиск аналогов

In [11]:
# Интерактивный поиск аналогов для конкретных товаров

def interactive_search(query: str, engine_name: str = 'hybrid', top_k: int = 10):
    """
    Интерактивный поиск аналогов с детальным выводом
    """
    print(f"🔍 Поиск аналогов для: '{query}'")
    print(f"🔧 Движок: {engine_name}")
    print(f"📊 Количество результатов: {top_k}")
    print("-" * 80)
    
    # Предобработка запроса
    if final_preprocess_function:
        processed_query = final_preprocess_function(query)
        print(f"📝 Обработанный запрос: '{processed_query}'")
        print("-" * 80)
    
    # Поиск аналогов
    results = search_analogs(query, engine_name, top_k)
    
    if results:
        print(f"✅ Найдено {len(results)} аналогов:")
        print()
        
        for i, result in enumerate(results, 1):
            original_name = result.get('original_name', '')
            score = result.get('combined_score', result.get('similarity_score', 0))
            
            # Анализ схожести
            similarity_analysis = analyze_similarity(query, original_name)
            
            print(f"{i}. {original_name}")
            print(f"   🎯 Поисковый скор: {score:.3f}")
            print(f"   📊 Схожесть: {similarity_analysis['similarity_score']:.3f}")
            print(f"   🔗 Тип связи: {similarity_analysis['relation_type']}")
            
            # Дополнительная информация
            row_data = result.get('row_data', {})
            if row_data:
                category = row_data.get('Группа', row_data.get('ВидНоменклатуры', ''))
                code = row_data.get('Код', '')
                if category:
                    print(f"   📂 Категория: {category}")
                if code:
                    print(f"   🏷️ Код: {code}")
            print()
    else:
        print("❌ Аналоги не найдены")

# Примеры поиска
print("🎯 Примеры интерактивного поиска:")
print("Используйте функцию interactive_search() для поиска аналогов")
print()
print("Примеры запросов:")
if len(processed_data) > 0:
    sample_names = processed_data[main_name_column].sample(3).tolist()
    for i, name in enumerate(sample_names, 1):
        print(f"{i}. interactive_search('{name[:50]}...', 'hybrid', 5)")
else:
    print("1. interactive_search('Болт М10×50 ГОСТ 7798-70', 'hybrid', 5)")
    print("2. interactive_search('Двигатель асинхронный 4кВт', 'semantic', 5)")
    print("3. interactive_search('Насос центробежный', 'fuzzy', 5)")

🎯 Примеры интерактивного поиска:
Используйте функцию interactive_search() для поиска аналогов

Примеры запросов:
1. interactive_search('Элемент фильтрующий Fleetguard HF51090A...', 'hybrid', 5)
2. interactive_search('Баллон для кислорода 40-150У ГОСТ 949-73...', 'hybrid', 5)
3. interactive_search('Ограничитель нагрузки DE-1000 13184-01...', 'hybrid', 5)


## 6. Заключение и инструкции

In [12]:
# Итоговая информация и инструкции
print("🎯 PRODUCTION СИСТЕМА ПОИСКА АНАЛОГОВ")
print("=" * 60)

print(f"\n📊 Статус системы:")
print(f"   Модель загружена: {'✅' if 'model_version' in locals() else '❌'}")
print(f"   Версия модели: {model_version if 'model_version' in locals() else 'N/A'}")
print(f"   Данные загружены: {'✅' if len(processed_data) > 0 else '❌'}")
print(f"   Записей в базе: {len(processed_data) if 'processed_data' in locals() else 0}")
print(f"   Поисковые движки: {', '.join(available_engines.keys()) if available_engines else 'Недоступны'}")
print(f"   Предобработка: {'✅' if final_preprocess_function else '❌'}")

print(f"\n🔧 Доступные функции:")
print(f"   • search_analogs(query, engine, top_k) - поиск аналогов")
print(f"   • interactive_search(query, engine, top_k) - интерактивный поиск")
print(f"   • analyze_similarity(query, candidate) - анализ схожести")
print(f"   • create_analog_analysis_report(sample_size) - создание отчета")

print(f"\n📋 Структура Excel отчета:")
print(f"   • Raw_Name - Оригинальное наименование")
print(f"   • Cleaned_Name - Очищенное наименование")
print(f"   • Lemmatized_Name - Лемматизированное наименование")
print(f"   • Normalized_Name - Нормализованное наименование")
print(f"   • Candidate_Name - Найденный аналог")
print(f"   • Similarity_Score - Оценка схожести (0-1)")
print(f"   • Relation_Type - Тип связи (дубль/аналог/похожий)")
print(f"   • Suggested_Category - Рекомендованная категория")
print(f"   • Final_Decision - Финальное решение")
print(f"   • Comment - Комментарий системы")

print(f"\n🚀 Примеры использования:")
print(f"\n# Поиск аналогов для конкретного товара")
print(f"results = search_analogs('Болт М10×50 ГОСТ 7798-70', 'hybrid', 5)")
print(f"\n# Интерактивный поиск с детальным выводом")
print(f"interactive_search('Двигатель асинхронный 4кВт', 'semantic', 10)")
print(f"\n# Создание полного отчета")
print(f"report_df = create_analog_analysis_report(sample_size=500)")

print(f"\n💡 Рекомендации:")
print(f"   • Используйте 'hybrid' движок для лучших результатов")
print(f"   • Начните с небольшой выборки для тестирования")
print(f"   • Проверяйте результаты с высокой схожестью на дубликаты")
print(f"   • Используйте интерактивный поиск для отладки")

if 'excel_path' in locals():
    print(f"\n📄 Последний созданный отчет: {excel_path}")

print(f"\n✅ Система готова к использованию!")
print(f"🕐 Время загрузки: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

🎯 PRODUCTION СИСТЕМА ПОИСКА АНАЛОГОВ

📊 Статус системы:
   Модель загружена: ✅
   Версия модели: v20250718_132401
   Данные загружены: ✅
   Записей в базе: 130303
   Поисковые движки: fuzzy, semantic, hybrid
   Предобработка: ❌

🔧 Доступные функции:
   • search_analogs(query, engine, top_k) - поиск аналогов
   • interactive_search(query, engine, top_k) - интерактивный поиск
   • analyze_similarity(query, candidate) - анализ схожести
   • create_analog_analysis_report(sample_size) - создание отчета

📋 Структура Excel отчета:
   • Raw_Name - Оригинальное наименование
   • Cleaned_Name - Очищенное наименование
   • Lemmatized_Name - Лемматизированное наименование
   • Normalized_Name - Нормализованное наименование
   • Candidate_Name - Найденный аналог
   • Similarity_Score - Оценка схожести (0-1)
   • Relation_Type - Тип связи (дубль/аналог/похожий)
   • Suggested_Category - Рекомендованная категория
   • Final_Decision - Финальное решение
   • Comment - Комментарий системы

🚀 Примеры ис